In [ ]:
# First, completely remove everything
!pip uninstall torch torchvision torchaudio torch-scatter torch-sparse torch-geometric -y

# Install a compatible PyTorch version that has pre-compiled wheels
!pip install torch==2.4.0+cu124 torchvision==0.19.0+cu124 torchaudio==2.4.0+cu124 --index-url https://download.pytorch.org/whl/cu124

# Install PyG libraries for this specific version
!pip install torch-scatter torch-sparse torch-geometric -f https://data.pyg.org/whl/torch-2.4.0+cu124.html

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 127.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 79.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.4/883.4 kB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8

In [ ]:
# ✅ OpenFDA (GraphML) + sections — R-GCN with Type-Aware, Filtered Negative Sampling (Train + Eval) — Single Cell

import os, random, pickle
from collections import defaultdict

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.nn import RGCNConv
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

import networkx as nx

# ----------------------------
# 0) Repro + Device
# ----------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ====================================================================================
# 1) Load Graph from GRAPHML (or PKL fallback) and normalize attributes
# ====================================================================================
# 👉 Set this to your GraphML path (example from your logs):
# graph_path = "/Users/ganeshkumarboini/Downloads/drug_data_kg_rebuilt_normalized.graphml"
graph_path = "/content/drive/MyDrive/drug_data_kg_rebuilt_normalized.graphml"

assert os.path.exists(graph_path), f"Path does not exist: {graph_path}"

def load_graph(path: str):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".graphml":
        # Read GraphML
        print(f"📥 Loading GraphML from: {path}")
        G0 = nx.read_graphml(path)  # may return Graph/DiGraph/MultiGraph/MultiDiGraph
        # Coerce to MultiDiGraph to preserve multi-edges + direction
        H = nx.MultiDiGraph()
        H.add_nodes_from(G0.nodes(data=True))
        if G0.is_multigraph():
            for u, v, k, data in G0.edges(keys=True, data=True):
                H.add_edge(u, v, key=k, **(data or {}))
        else:
            # If undirected, we still put edges in a DiGraph form; GraphML may not carry direction flags per-edge.
            for u, v, data in G0.edges(data=True):
                H.add_edge(u, v, **(data or {}))
        return H
    elif ext in [".pkl", ".pickle"]:
        print(f"📥 Loading pickle from: {path}")
        with open(path, "rb") as f:
            return pickle.load(f)
    else:
        raise ValueError(f"Unsupported file extension: {ext}")

G = load_graph(graph_path)

# --- Patch node attributes: prefer 'node_type'; fallback to 'type'/'category'/'kind' ---
for n, data in G.nodes(data=True):
    if "node_type" not in data:
        if "type" in data:
            data["node_type"] = data["type"]
        elif "category" in data:
            data["node_type"] = data["category"]
        elif "kind" in data:
            data["node_type"] = data["kind"]
        else:
            data["node_type"] = "unknown"

# --- Patch edge attributes: prefer 'relation'; fallback to 'type'/'label' ---
for u, v, attr in G.edges(data=True):
    if "relation" not in attr:
        if "type" in attr:
            attr["relation"] = attr["type"]
        elif "label" in attr:
            attr["relation"] = attr["label"]
        else:
            attr["relation"] = "unknown"

print(f"✅ Loaded graph with {len(G.nodes)} nodes and {len(G.edges)} edges.")
print(f"🔎 Example node attrs keys: {list(next(iter(G.nodes(data=True)))[1].keys()) if len(G)>0 else []}")
print(f"🔎 Example edge attrs keys: {list(next(iter(G.edges(data=True)))[2].keys()) if G.number_of_edges()>0 else []}")

# ====================================================================================
# 2) Build type maps and edge maps (typed remap to homogeneous ids)
# ====================================================================================
node_type_map = defaultdict(list)      # {ntype: [original node ids in this type order]}
node_id_map = {}                       # {original node id: (ntype, local_idx)}
node_idx_by_type = defaultdict(dict)   # {ntype: {original id: local_idx}}

for nid, data in G.nodes(data=True):
    ntype = data.get("node_type", "unknown")
    idx = len(node_type_map[ntype])
    node_type_map[ntype].append(nid)
    node_id_map[nid] = (ntype, idx)
    node_idx_by_type[ntype][nid] = idx

edge_type_map = defaultdict(list)      # {(src_type, relation, dst_type): [(src_local, dst_local), ...]}
for src, dst, attr in G.edges(data=True):
    rel = attr.get("relation", "unknown")
    if src not in node_id_map or dst not in node_id_map:
        continue
    src_t, src_i = node_id_map[src]
    dst_t, dst_i = node_id_map[dst]
    edge_type_map[(src_t, rel, dst_t)].append((src_i, dst_i))

print(f"✅ Node types: {list(node_type_map.keys())}")
print(f"✅ Total edge types: {len(edge_type_map)}")

# ====================================================================================
# 3) Homogeneous graph remap + per-relation splits
# ====================================================================================
type_offsets = {}
global_id_map = {}
offset = 0
for ntype, nodes in node_type_map.items():
    type_offsets[ntype] = offset
    for local_idx in range(len(nodes)):
        global_id_map[(ntype, local_idx)] = offset + local_idx
    offset += len(nodes)

num_nodes = offset
edge_type_keys = list(edge_type_map.keys())            # index i -> (src_type, rel, dst_type)
rel2id = {etype: i for i, etype in enumerate(edge_type_keys)}
num_relations = len(edge_type_keys)

all_src, all_dst, all_rel = [], [], []
edge_type_splits = {}

for etype, edges in edge_type_map.items():
    src_type, _, dst_type = etype
    if len(edges) < 5:
        continue
    perm = torch.randperm(len(edges))
    num_train = int(0.8 * len(edges))
    num_val = int(0.1 * len(edges))
    train_edges_local = [edges[i] for i in perm[:num_train]]
    val_edges_local   = [edges[i] for i in perm[num_train:num_train+num_val]]
    test_edges_local  = [edges[i] for i in perm[num_train+num_val:]]

    r = rel2id[etype]
    for (s_l, d_l) in train_edges_local:
        all_src.append(type_offsets[src_type] + s_l)
        all_dst.append(type_offsets[dst_type] + d_l)
        all_rel.append(r)

    edge_type_splits[etype] = {
        "val": torch.tensor([
            [type_offsets[src_type] + s for s, d in val_edges_local],
            [type_offsets[dst_type] + d for s, d in val_edges_local]
        ], dtype=torch.long),
        "test": torch.tensor([
            [type_offsets[src_type] + s for s, d in test_edges_local],
            [type_offsets[dst_type] + d for s, d in test_edges_local]
        ], dtype=torch.long)
    }

edge_index = torch.tensor([all_src, all_dst], dtype=torch.long)
edge_type = torch.tensor(all_rel, dtype=torch.long)
print(f"Homogeneous nodes: {num_nodes}, edges used for training: {edge_index.size(1)}")

# Move static tensors to device ONCE
edge_index = edge_index.to(device)
edge_type  = edge_type.to(device)
node_ids   = torch.arange(num_nodes, device=device)

# Existing edges per relation (for filtered negatives) — keep as LOCAL pairs
existing_edges_per_rel = {etype: set(pairs) for etype, pairs in edge_type_map.items()}
def relid_to_etype(rel_id: int):
    return edge_type_keys[rel_id]

# ====================================================================================
# 4) Model
# ====================================================================================
class RGCN(torch.nn.Module):
    def __init__(self, num_nodes, num_relations, emb_dim=64, hidden_dim=64, out_dim=32):
        super().__init__()
        self.emb = torch.nn.Embedding(num_nodes, emb_dim)
        self.conv1 = RGCNConv(emb_dim, hidden_dim, num_relations=num_relations)
        self.conv2 = RGCNConv(hidden_dim, out_dim, num_relations=num_relations)

    def forward(self, node_ids, edge_index, edge_type):
        x = self.emb(node_ids)
        x = self.conv1(x, edge_index, edge_type)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_type)
        return x

def edge_scores(z, eidx):
    return (z[eidx[0]] * z[eidx[1]]).sum(dim=-1).sigmoid()

# ====================================================================================
# 5) Type-aware, relation-aware, filtered negative sampling
# ====================================================================================
def sample_neg_edges_filtered_batch(src_g, dst_g, rel_ids, k_neg,
                                    node_type_map, type_offsets, existing_edges_per_rel):
    assert src_g.shape == dst_g.shape == rel_ids.shape
    neg_edges = []
    type_sizes = {ntype: len(nodes) for ntype, nodes in node_type_map.items()}

    for s_glob, d_glob, r_id in zip(src_g.tolist(), dst_g.tolist(), rel_ids.tolist()):
        etype = relid_to_etype(r_id)   # (src_type, rel, dst_type)
        src_type, _, dst_type = etype
        s_local = s_glob - type_offsets[src_type]
        d_local = d_glob - type_offsets[dst_type]
        existing = existing_edges_per_rel[etype]
        src_len = type_sizes[src_type]; dst_len = type_sizes[dst_type]

        for _ in range(k_neg):
            if random.random() < 0.5:  # corrupt head
                # try random heads until a non-existing pair is found
                while True:
                    new_s_local = random.randrange(src_len)
                    if (new_s_local, d_local) not in existing:
                        neg_edges.append((type_offsets[src_type] + new_s_local, d_glob))
                        break
            else:                      # corrupt tail
                while True:
                    new_d_local = random.randrange(dst_len)
                    if (s_local, new_d_local) not in existing:
                        neg_edges.append((s_glob, type_offsets[dst_type] + new_d_local))
                        break

    if not neg_edges:
        return torch.empty((2, 0), dtype=torch.long, device=src_g.device)
    return torch.tensor(neg_edges, dtype=torch.long, device=src_g.device).T  # (2, K*B)

@torch.no_grad()
def evaluate_filtered(z, pos_eidx, rel_id, k_neg,
                      node_type_map, type_offsets, existing_edges_per_rel, device):
    if pos_eidx.numel() == 0:
        return float("nan"), float("nan"), float("nan"), float("nan")
    N = pos_eidx.size(1)
    src_g = pos_eidx[0].to(device)
    dst_g = pos_eidx[1].to(device)
    rel_ids = torch.full((N,), rel_id, dtype=torch.long, device=device)

    neg_eidx = sample_neg_edges_filtered_batch(
        src_g, dst_g, rel_ids, k_neg, node_type_map, type_offsets, existing_edges_per_rel
    )

    y_true = torch.cat([torch.ones(N, device=device),
                        torch.zeros(neg_eidx.size(1), device=device)])
    scores = torch.cat([edge_scores(z, pos_eidx.to(device)),
                        edge_scores(z, neg_eidx)], dim=0)

    y, s = y_true.detach().cpu().numpy(), scores.detach().cpu().numpy()
    auc = roc_auc_score(y, s)
    preds = (s > 0.5).astype(int)
    precision = precision_score(y, preds)
    recall = recall_score(y, preds)
    f1 = f1_score(y, preds)
    return auc, precision, recall, f1

# ====================================================================================
# 6) Training setup
# ====================================================================================
EPOCHS = 30
LR = 0.01
BATCH_SIZE = 50_000
K_NEG = 1  # negatives per positive

model = RGCN(num_nodes, num_relations).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

import torch.utils.data as tud
train_edges = torch.tensor([all_src, all_dst], dtype=torch.long)
train_rels  = torch.tensor(all_rel, dtype=torch.long)
train_dataset = tud.TensorDataset(train_edges[0], train_edges[1], train_rels)
train_loader = tud.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)

# ====================================================================================
# 7) Train (single forward/backward per batch)
# ====================================================================================
print("\n🔁 Training unified R-GCN (type-aware, filtered negatives)...")
for epoch in range(1, EPOCHS + 1):
    model.train()
    epoch_loss = 0.0

    for src_batch, dst_batch, rel_batch in train_loader:
        src_batch = src_batch.to(device)
        dst_batch = dst_batch.to(device)
        rel_batch = rel_batch.to(device)

        optimizer.zero_grad()

        # Compute embeddings (forward) using full graph edges each batch
        z = model(node_ids, edge_index, edge_type)

        # Positives
        pos_eidx = torch.stack([src_batch, dst_batch])  # (2, B)

        # Type-aware, relation-aware, filtered negatives
        neg_eidx = sample_neg_edges_filtered_batch(
            src_batch, dst_batch, rel_batch, K_NEG,
            node_type_map, type_offsets, existing_edges_per_rel
        )

        # Loss
        pos_loss = -torch.log(edge_scores(z, pos_eidx) + 1e-15).mean()
        if neg_eidx.size(1) > 0:
            neg_loss = -torch.log(1.0 - edge_scores(z, neg_eidx) + 1e-15).mean()
            loss = pos_loss + neg_loss
        else:
            loss = pos_loss

        loss.backward()
        optimizer.step()

        epoch_loss += float(loss.detach().cpu())

    print(f"Epoch {epoch:02d} | Total Loss: {epoch_loss:.4f}")

# ====================================================================================
# 8) Final evaluation (per relation + overall)
# ====================================================================================
print("\n📊 Final evaluation per edge type (filtered negatives):")
model.eval()
with torch.no_grad():
    z = model(node_ids, edge_index, edge_type)

results = []
all_pos_eidx_list, all_rel_ids_list = [], []

for etype, splits in edge_type_splits.items():
    rel_id = rel2id[etype]
    test_edges = splits["test"]  # (2, N)
    if test_edges.size(1) == 0:
        continue

    auc, p, r, f1 = evaluate_filtered(
        z, test_edges, rel_id, K_NEG,
        node_type_map, type_offsets, existing_edges_per_rel, device
    )
    results.append((etype, auc, p, r, f1))
    all_pos_eidx_list.append(test_edges)
    all_rel_ids_list.append(torch.full((test_edges.size(1),), rel_id, dtype=torch.long))

    print(f"{etype}: AUC={auc:.4f}, P={p:.3f}, R={r:.3f}, F1={f1:.3f}")

if all_pos_eidx_list:
    all_pos_eidx = torch.cat(all_pos_eidx_list, dim=1)
    all_rel_ids = torch.cat(all_rel_ids_list, dim=0).to(device)

    neg_eidx_overall = sample_neg_edges_filtered_batch(
        all_pos_eidx[0].to(device), all_pos_eidx[1].to(device),
        all_rel_ids, K_NEG, node_type_map, type_offsets, existing_edges_per_rel
    )

    y_true_overall = torch.cat([torch.ones(all_pos_eidx.size(1), device=device),
                                torch.zeros(neg_eidx_overall.size(1), device=device)])
    scores_overall = torch.cat([edge_scores(z, all_pos_eidx.to(device)),
                                edge_scores(z, neg_eidx_overall)], dim=0)

    y_cpu = y_true_overall.detach().cpu().numpy()
    s_cpu = scores_overall.detach().cpu().numpy()
    overall_auc = roc_auc_score(y_cpu, s_cpu)
    overall_preds = (s_cpu > 0.5).astype(int)
    overall_p = precision_score(y_cpu, overall_preds)
    overall_r = recall_score(y_cpu, overall_preds)
    overall_f1 = f1_score(y_cpu, overall_preds)

    print("\n🌍 Overall Performance (filtered negatives):")
    print(f"Overall AUC = {overall_auc:.4f}")
    print(f"Overall Precision = {overall_p:.3f}")
    print(f"Overall Recall = {overall_r:.3f}")
    print(f"Overall F1 Score = {overall_f1:.3f}")
else:
    print("\nNo test edges available for evaluation.")


Using device: cuda
📥 Loading GraphML from: /content/drive/MyDrive/drug_data_kg_rebuilt_normalized.graphml
✅ Loaded graph with 100495 nodes and 160229 edges.
🔎 Example node attrs keys: ['type', 'node_type']
🔎 Example edge attrs keys: ['relationship', 'orig_relationship', 'relation']
✅ Node types: ['drug', 'active_ingredient', 'indications_and_usage', 'dosage_and_administration', 'warnings', 'pregnancy_or_breast_feeding', 'keep_out_of_reach_of_children', 'purpose', 'storage_and_handling', 'brand', 'do_not_use', 'ask_doctor', 'inactive_ingredient', 'stop_use', 'ask_doctor_or_pharmacist', 'overdose_warning', 'drug_interactions']
✅ Total edge types: 287
Homogeneous nodes: 100495, edges used for training: 127935

🔁 Training unified R-GCN (type-aware, filtered negatives)...


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 24.12 MiB is free. Process 17690 has 14.71 GiB memory in use. Of the allocated memory 13.76 GiB is allocated by PyTorch, and 852.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# === GraphML → R-GCN with NeighborLoader mini-batching + AMP + batched inference (type-aware, filtered negatives) ===
# Works with GraphML input. Keeps your training/eval structure, adds robust attr normalization.

import os, gc, random, pickle
from collections import defaultdict

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch_geometric.nn import RGCNConv
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
import networkx as nx

# ----------------------------
# 0) Repro + Device + GC
# ----------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# (Optional) helps reduce CUDA fragmentation on long runs
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gc.collect()
if device.type == "cuda":
    torch.cuda.empty_cache()
print("Using device:", device)

# ----------------------------
# 1) Load Graph from GRAPHML (with attr normalization)
# ----------------------------
# Set your GraphML path here:
graph_path = "/content/drive/MyDrive/drug_data_kg_rebuilt_normalized.graphml"
assert os.path.exists(graph_path), f"Path does not exist: {graph_path}"

print(f"📥 Loading GraphML from: {graph_path}")
G0 = nx.read_graphml(graph_path)

# Coerce to MultiDiGraph to preserve multi-edges + direction
G = nx.MultiDiGraph()
G.add_nodes_from(G0.nodes(data=True))
if G0.is_multigraph():
    for u, v, k, data in G0.edges(keys=True, data=True):
        G.add_edge(u, v, key=k, **(data or {}))
else:
    for u, v, data in G0.edges(data=True):
        G.add_edge(u, v, **(data or {}))

# --- Normalize node attributes: prefer 'node_type', fallback to 'type'/'category'/'kind' ---
for n, data in G.nodes(data=True):
    if "node_type" not in data:
        if "type" in data:        data["node_type"] = data["type"]
        elif "category" in data:  data["node_type"] = data["category"]
        elif "kind" in data:      data["node_type"] = data["kind"]
        else:                     data["node_type"] = "unknown"

# --- Normalize edge attributes: prefer 'relation', fallback to 'relationship'/'type'/'label' ---
for u, v, attr in G.edges(data=True):
    if "relation" not in attr:
        if   "relationship" in attr: attr["relation"] = attr["relationship"]
        elif "type" in attr:         attr["relation"] = attr["type"]
        elif "label" in attr:        attr["relation"] = attr["label"]
        else:                        attr["relation"] = "unknown"

print(f"✅ Loaded graph with {len(G.nodes)} nodes and {len(G.edges)} edges.")

# ----------------------------
# 2) Build type maps and edge maps
# ----------------------------
node_type_map = defaultdict(list)      # {ntype: [original node ids in this type order]}
node_id_map = {}                       # {original node id: (ntype, local_idx)}
node_idx_by_type = defaultdict(dict)   # {ntype: {original id: local_idx}}

for nid, data in G.nodes(data=True):
    ntype = data.get("node_type", "unknown")
    idx = len(node_type_map[ntype])
    node_type_map[ntype].append(nid)
    node_id_map[nid] = (ntype, idx)
    node_idx_by_type[ntype][nid] = idx

edge_type_map = defaultdict(list)      # {(src_type, relation, dst_type): [(src_local, dst_local), ...]}
for src, dst, attr in G.edges(data=True):
    rel = attr.get("relation", "unknown")
    if src not in node_id_map or dst not in node_id_map:
        continue
    src_t, src_i = node_id_map[src]
    dst_t, dst_i = node_id_map[dst]
    edge_type_map[(src_t, rel, dst_t)].append((src_i, dst_i))

print(f"✅ Node types: {list(node_type_map.keys())}")
print(f"✅ Total edge types: {len(edge_type_map)}")

# ----------------------------
# 3) Homogeneous graph remap + per-relation splits
# ----------------------------
type_offsets = {}
global_id_map = {}
offset = 0
for ntype, nodes in node_type_map.items():
    type_offsets[ntype] = offset
    for local_idx in range(len(nodes)):
        global_id_map[(ntype, local_idx)] = offset + local_idx
    offset += len(nodes)

num_nodes = offset
edge_type_keys = list(edge_type_map.keys())            # index i -> (src_type, rel, dst_type)
rel2id = {etype: i for i, etype in enumerate(edge_type_keys)}
num_relations = len(edge_type_keys)

all_src, all_dst, all_rel = [], [], []
edge_type_splits = {}

for etype, edges in edge_type_map.items():
    src_type, _, dst_type = etype
    if len(edges) < 5:
        continue
    perm = torch.randperm(len(edges))
    num_train = int(0.8 * len(edges))
    num_val = int(0.1 * len(edges))
    train_edges_local = [edges[i] for i in perm[:num_train]]
    val_edges_local   = [edges[i] for i in perm[num_train:num_train+num_val]]
    test_edges_local  = [edges[i] for i in perm[num_train+num_val:]]

    r = rel2id[etype]
    for (s_l, d_l) in train_edges_local:
        all_src.append(type_offsets[src_type] + s_l)
        all_dst.append(type_offsets[dst_type] + d_l)
        all_rel.append(r)

    edge_type_splits[etype] = {
        "val": torch.tensor([
            [type_offsets[src_type] + s for s, d in val_edges_local],
            [type_offsets[dst_type] + d for s, d in val_edges_local]
        ], dtype=torch.long),
        "test": torch.tensor([
            [type_offsets[src_type] + s for s, d in test_edges_local],
            [type_offsets[dst_type] + d for s, d in test_edges_local]
        ], dtype=torch.long)
    }

edge_index_cpu = torch.tensor([all_src, all_dst], dtype=torch.long)  # CPU for loaders
edge_type_cpu  = torch.tensor(all_rel, dtype=torch.long)

print(f"Homogeneous nodes: {num_nodes}, edges used for training: {edge_index_cpu.size(1)}")

# ----------------------------
# 4) Existing edges per relation (for filtered negatives)
# ----------------------------
existing_edges_per_rel = {etype: set(pairs) for etype, pairs in edge_type_map.items()}
def relid_to_etype(rel_id: int):
    return edge_type_keys[rel_id]

# ----------------------------
# 5) Model (R-GCN with num_bases) + helpers
# ----------------------------
class RGCN(nn.Module):
    def __init__(self, num_nodes, num_relations, emb_dim=64, hidden_dim=128, out_dim=32, num_bases=30):
        super().__init__()
        self.emb = nn.Embedding(num_nodes, emb_dim)
        self.conv1 = RGCNConv(emb_dim, hidden_dim, num_relations=num_relations, num_bases=num_bases)
        self.conv2 = RGCNConv(hidden_dim, out_dim, num_relations=num_relations, num_bases=num_bases)

    def forward(self, node_ids, edge_index, edge_type):
        # node_ids: global ids present in this sampled subgraph (order matches subgraph nodes)
        x = self.emb(node_ids)
        x = self.conv1(x, edge_index, edge_type)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_type)
        return x

def edge_scores(z, eidx):
    return (z[eidx[0]] * z[eidx[1]]).sum(dim=-1).sigmoid()

# ----------------------------
# 6) Negative sampling & evaluation (filtered, type-aware)
# ----------------------------
def sample_neg_edges_filtered_batch(src_g, dst_g, rel_ids, k_neg,
                                    node_type_map, type_offsets, existing_edges_per_rel):
    assert src_g.shape == dst_g.shape == rel_ids.shape
    neg_edges = []
    type_sizes = {ntype: len(nodes) for ntype, nodes in node_type_map.items()}

    for s_glob, d_glob, r_id in zip(src_g.tolist(), dst_g.tolist(), rel_ids.tolist()):
        etype = relid_to_etype(r_id)   # (src_type, rel, dst_type)
        src_type, _, dst_type = etype
        s_local = s_glob - type_offsets[src_type]
        d_local = d_glob - type_offsets[dst_type]
        existing = existing_edges_per_rel[etype]
        src_len = type_sizes[src_type]; dst_len = type_sizes[dst_type]

        for _ in range(k_neg):
            if random.random() < 0.5:  # corrupt head
                while True:
                    new_s_local = random.randrange(src_len)
                    if (new_s_local, d_local) not in existing:
                        neg_edges.append((type_offsets[src_type] + new_s_local, d_glob))
                        break
            else:                      # corrupt tail
                while True:
                    new_d_local = random.randrange(dst_len)
                    if (s_local, new_d_local) not in existing:
                        neg_edges.append((s_glob, type_offsets[dst_type] + new_d_local))
                        break

    if not neg_edges:
        return torch.empty((2, 0), dtype=torch.long, device=src_g.device)
    return torch.tensor(neg_edges, dtype=torch.long, device=src_g.device).T  # (2, K*B)

@torch.no_grad()
def evaluate_filtered(z, pos_eidx, rel_id, k_neg,
                      node_type_map, type_offsets, existing_edges_per_rel, device):
    if pos_eidx.numel() == 0:
        return float("nan"), float("nan"), float("nan"), float("nan")
    N = pos_eidx.size(1)
    src_g = pos_eidx[0].to(device, non_blocking=True)
    dst_g = pos_eidx[1].to(device, non_blocking=True)
    rel_ids = torch.full((N,), rel_id, dtype=torch.long, device=device)

    neg_eidx = sample_neg_edges_filtered_batch(
        src_g, dst_g, rel_ids, k_neg, node_type_map, type_offsets, existing_edges_per_rel
    )

    y_true = torch.cat([torch.ones(N, device=device),
                        torch.zeros(neg_eidx.size(1), device=device)])
    scores = torch.cat([edge_scores(z, pos_eidx.to(device, non_blocking=True)),
                        edge_scores(z, neg_eidx)], dim=0)

    y, s = y_true.detach().cpu().numpy(), scores.detach().cpu().numpy()
    auc = roc_auc_score(y, s)
    preds = (s > 0.5).astype(int)
    precision = precision_score(y, preds)
    recall = recall_score(y, preds)
    f1 = f1_score(y, preds)
    return auc, precision, recall, f1

# ----------------------------
# 7) Training setup & PyG Data + NeighborLoader
# ----------------------------
EPOCHS = 30
LR = 0.01
K_NEG = 1  # negatives per positive

# Build a PyG Data object (no node features; embeddings live in the model)
data_pyg = Data(num_nodes=num_nodes, edge_index=edge_index_cpu, edge_type=edge_type_cpu)

# Use unique training source nodes as target nodes for sampling
train_src_tensor = torch.tensor(all_src, dtype=torch.long)
train_src_unique = torch.unique(train_src_tensor)

# NeighborLoader settings: tune NUM_NEIGHBORS and NBATCH_SIZE to taste
NUM_NEIGHBORS = [15, 10]   # sample neighbors per R-GCN layer
NBATCH_SIZE = 4096         # number of target nodes per mini-batch (reduce if OOM)
NUM_WORKERS = 4

train_loader = NeighborLoader(
    data_pyg,
    num_neighbors=NUM_NEIGHBORS,
    batch_size=NBATCH_SIZE,
    input_nodes=train_src_unique,  # sample neighborhoods around training source nodes
    shuffle=True,
    num_workers=NUM_WORKERS
)

# ----------------------------
# 8) Model, optimizer, AMP scaler
# ----------------------------
model = RGCN(num_nodes, num_relations, emb_dim=64, hidden_dim=128, out_dim=32, num_bases=30).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scaler = GradScaler(enabled=(device.type == "cuda"))

# Pre-build global training tensors on DEVICE once to avoid re-allocating every batch
src_glob_all = torch.tensor(all_src, dtype=torch.long, device=device)
dst_glob_all = torch.tensor(all_dst, dtype=torch.long, device=device)
rels_all     = torch.tensor(all_rel, dtype=torch.long, device=device)

# ----------------------------
# 9) Training loop using NeighborLoader (mini-batched subgraphs)
# ----------------------------
print("\n🔁 Training with NeighborLoader mini-batches + AMP ...")
for epoch in range(1, EPOCHS + 1):
    model.train()
    epoch_loss = 0.0
    seen_pos = 0

    for batch in train_loader:
        batch = batch.to(device, non_blocking=True)

        # global ids of nodes in this batch (same order as local nodes)
        n_id = batch.n_id  # [num_nodes_in_batch], already on device

        # global->local mapping for this batch
        global2local = -torch.ones(num_nodes, dtype=torch.long, device=device)
        global2local[n_id] = torch.arange(n_id.size(0), device=device)

        # Which training positives have both endpoints inside this batch?
        in_src = global2local[src_glob_all]  # -1 if source not in batch
        in_dst = global2local[dst_glob_all]  # -1 if dest not in batch
        mask_in_batch = (in_src >= 0) & (in_dst >= 0)

        pos_idxs = torch.nonzero(mask_in_batch, as_tuple=False).view(-1)
        if pos_idxs.numel() == 0:
            del batch, n_id, global2local, in_src, in_dst, mask_in_batch, pos_idxs
            if device.type == "cuda": torch.cuda.empty_cache()
            continue

        # Optional sub-sample positives per batch to cap compute
        MAX_POS_PER_BATCH = 8192
        if pos_idxs.numel() > MAX_POS_PER_BATCH:
            perm = torch.randperm(pos_idxs.numel(), device=device)[:MAX_POS_PER_BATCH]
            pos_idxs = pos_idxs[perm]

        # Local indices for positives (local to this batch)
        local_src = global2local[src_glob_all[pos_idxs]]
        local_dst = global2local[dst_glob_all[pos_idxs]]
        rel_ids_for_pos = rels_all[pos_idxs]

        pos_eidx_local  = torch.stack([local_src, local_dst], dim=0)                   # (2, P)
        pos_eidx_global = torch.stack([src_glob_all[pos_idxs], dst_glob_all[pos_idxs]], dim=0)  # (2, P)

        optimizer.zero_grad(set_to_none=True)

        with autocast(enabled=(device.type == "cuda")):
            z_batch = model(n_id, batch.edge_index, batch.edge_type)  # [n_in_batch, out_dim]

            # Typed, filtered negatives in GLOBAL id space
            neg_eidx_global = sample_neg_edges_filtered_batch(
                pos_eidx_global[0], pos_eidx_global[1], rel_ids_for_pos, K_NEG,
                node_type_map, type_offsets, existing_edges_per_rel
            )

            # Map negatives into local indices; keep those inside sampled subgraph
            if neg_eidx_global.numel() > 0:
                neg_src_local = global2local[neg_eidx_global[0]]
                neg_dst_local = global2local[neg_eidx_global[1]]
                neg_mask = (neg_src_local >= 0) & (neg_dst_local >= 0)
                if neg_mask.any():
                    neg_eidx_local = torch.stack([neg_src_local[neg_mask], neg_dst_local[neg_mask]], dim=0)
                else:
                    neg_eidx_local = torch.empty((2,0), dtype=torch.long, device=device)
            else:
                neg_eidx_local = torch.empty((2,0), dtype=torch.long, device=device)

            # Scores & loss (DistMult with sigmoid)
            pos_scores = (z_batch[pos_eidx_local[0]] * z_batch[pos_eidx_local[1]]).sum(dim=-1).sigmoid()
            pos_loss = -torch.log(pos_scores + 1e-15).mean()

            if neg_eidx_local.size(1) > 0:
                neg_scores = (z_batch[neg_eidx_local[0]] * z_batch[neg_eidx_local[1]]).sum(dim=-1).sigmoid()
                neg_loss = -torch.log(1.0 - neg_scores + 1e-15).mean()
                loss = pos_loss + neg_loss
            else:
                loss = pos_loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += float(loss.detach().cpu())
        seen_pos += pos_eidx_local.size(1)

        # Free batch-level tensors
        del batch, n_id, global2local, in_src, in_dst, mask_in_batch, pos_idxs
        del local_src, local_dst, rel_ids_for_pos, pos_eidx_local, pos_eidx_global
        del z_batch, neg_eidx_global, neg_eidx_local, pos_scores, pos_loss, loss
        if device.type == "cuda":
            torch.cuda.empty_cache()

    print(f"Epoch {epoch:02d} | Loss: {epoch_loss:.4f} | Seen pos edges: {seen_pos}")

# ----------------------------
# 10) Batched inference: compute z for ALL nodes without OOM
# ----------------------------
print("\n🔎 Computing full-node embeddings (batched inference) ...")
model.eval()
OUT_DIM = 32  # same as model out_dim
z_all = torch.empty((num_nodes, OUT_DIM), dtype=torch.float32, device=device)

INFER_BATCH = 16384  # tune smaller if OOM during inference
infer_loader = NeighborLoader(
    Data(num_nodes=num_nodes, edge_index=edge_index_cpu, edge_type=edge_type_cpu),
    num_neighbors=[-1, -1],    # full 2-hop for both layers
    batch_size=INFER_BATCH,
    input_nodes=torch.arange(num_nodes),
    shuffle=False,
    num_workers=NUM_WORKERS
)

with torch.no_grad():
    for batch in infer_loader:
        batch = batch.to(device, non_blocking=True)
        n_id = batch.n_id
        z_batch = model(n_id, batch.edge_index, batch.edge_type)
        z_all[n_id] = z_batch
        del batch, z_batch, n_id
        if device.type == "cuda":
            torch.cuda.empty_cache()

# ----------------------------
# 11) Final evaluation (per relation + overall) using z_all
# ----------------------------
print("\n📊 Final evaluation per edge type (filtered negatives):")
results = []
all_pos_eidx_list, all_rel_ids_list = [], []

def edge_scores_all(z, eidx):
    return (z[eidx[0]] * z[eidx[1]]).sum(dim=-1).sigmoid()

for etype, splits in edge_type_splits.items():
    rel_id = rel2id[etype]
    test_edges = splits["test"]  # (2, N)
    if test_edges.size(1) == 0:
        continue

    auc, p, r, f1 = evaluate_filtered(
        z_all, test_edges, rel_id, K_NEG,
        node_type_map, type_offsets, existing_edges_per_rel, device
    )
    results.append((etype, auc, p, r, f1))
    all_pos_eidx_list.append(test_edges)
    all_rel_ids_list.append(torch.full((test_edges.size(1),), rel_id, dtype=torch.long))

    print(f"{etype}: AUC={auc:.4f}, P={p:.3f}, R={r:.3f}, F1={f1:.3f}")

if all_pos_eidx_list:
    all_pos_eidx = torch.cat(all_pos_eidx_list, dim=1)
    all_rel_ids = torch.cat(all_rel_ids_list, dim=0).to(device)

    neg_eidx_overall = sample_neg_edges_filtered_batch(
        all_pos_eidx[0].to(device, non_blocking=True),
        all_pos_eidx[1].to(device, non_blocking=True),
        all_rel_ids, K_NEG, node_type_map, type_offsets, existing_edges_per_rel
    )

    y_true_overall = torch.cat([torch.ones(all_pos_eidx.size(1), device=device),
                                torch.zeros(neg_eidx_overall.size(1), device=device)])
    scores_overall = torch.cat([edge_scores_all(z_all, all_pos_eidx.to(device, non_blocking=True)),
                                edge_scores_all(z_all, neg_eidx_overall)], dim=0)

    y_cpu = y_true_overall.detach().cpu().numpy()
    s_cpu = scores_overall.detach().cpu().numpy()
    overall_auc = roc_auc_score(y_cpu, s_cpu)
    overall_preds = (s_cpu > 0.5).astype(int)
    overall_p = precision_score(y_cpu, overall_preds)
    overall_r = recall_score(y_cpu, overall_preds)
    overall_f1 = f1_score(y_cpu, overall_preds)

    print("\n🌍 Overall Performance (filtered negatives):")
    print(f"Overall AUC = {overall_auc:.4f}")
    print(f"Overall Precision = {overall_p:.3f}")
    print(f"Overall Recall = {overall_r:.3f}")
    print(f"Overall F1 Score = {overall_f1:.3f}")
else:
    print("\nNo test edges available for evaluation.")


Using device: cuda
📥 Loading GraphML from: /content/drive/MyDrive/drug_data_kg_rebuilt_normalized.graphml
✅ Loaded graph with 100495 nodes and 160229 edges.
✅ Node types: ['drug', 'active_ingredient', 'indications_and_usage', 'dosage_and_administration', 'warnings', 'pregnancy_or_breast_feeding', 'keep_out_of_reach_of_children', 'purpose', 'storage_and_handling', 'brand', 'do_not_use', 'ask_doctor', 'inactive_ingredient', 'stop_use', 'ask_doctor_or_pharmacist', 'overdose_warning', 'drug_interactions']
✅ Total edge types: 287
Homogeneous nodes: 100495, edges used for training: 127935

🔁 Training with NeighborLoader mini-batches + AMP ...


/usr/local/lib/python3.12/dist-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/tmp/ipython-input-3778466115.py:274: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(devi

Epoch 01 | Loss: 31.9452 | Seen pos edges: 23052
Epoch 02 | Loss: 15.5384 | Seen pos edges: 22846
Epoch 03 | Loss: 12.4352 | Seen pos edges: 23092
Epoch 04 | Loss: 11.0944 | Seen pos edges: 22765
Epoch 05 | Loss: 10.4573 | Seen pos edges: 22956
Epoch 06 | Loss: 10.0183 | Seen pos edges: 22752
Epoch 07 | Loss: 9.7059 | Seen pos edges: 22912
Epoch 08 | Loss: 9.1485 | Seen pos edges: 22875
Epoch 09 | Loss: 9.2289 | Seen pos edges: 22826
Epoch 10 | Loss: 8.8611 | Seen pos edges: 22681
Epoch 11 | Loss: 8.7910 | Seen pos edges: 22818
Epoch 12 | Loss: 8.6300 | Seen pos edges: 22913
Epoch 13 | Loss: 8.4246 | Seen pos edges: 23017
Epoch 14 | Loss: 8.1609 | Seen pos edges: 23039
Epoch 15 | Loss: 8.2283 | Seen pos edges: 22826
Epoch 16 | Loss: 7.9806 | Seen pos edges: 23088
Epoch 17 | Loss: 8.0171 | Seen pos edges: 22912
Epoch 18 | Loss: 7.8431 | Seen pos edges: 23159
Epoch 19 | Loss: 7.6005 | Seen pos edges: 23121
Epoch 20 | Loss: 7.6930 | Seen pos edges: 22772
Epoch 21 | Loss: 7.4863 | Seen pos

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


('inactive_ingredient', 'warnings', 'warnings'): AUC=0.7200, P=0.571, R=0.800, F1=0.667
('inactive_ingredient', 'keep_out_of_reach_of_children', 'keep_out_of_reach_of_children'): AUC=0.3333, P=0.000, R=0.000, F1=0.000
('inactive_ingredient', 'dosage_and_administration', 'dosage_and_administration'): AUC=0.5000, P=0.500, R=0.250, F1=0.333
('brand', 'warnings', 'warnings'): AUC=0.4994, P=0.493, R=0.555, F1=0.522
('drug', 'stop_use', 'stop_use'): AUC=0.5074, P=0.537, R=0.329, F1=0.408
('drug', 'stop_use', 'warnings'): AUC=0.6341, P=0.583, R=0.667, F1=0.622
('drug', 'do_not_use', 'warnings'): AUC=0.3294, P=0.290, R=0.209, F1=0.243
('drug', 'keep_out_of_reach_of_children', 'warnings'): AUC=0.6167, P=0.556, R=0.653, F1=0.601
('brand', 'active_ingredient', 'brand'): AUC=0.2500, P=0.333, R=0.500, F1=0.400
('brand', 'active_ingredient', 'active_ingredient'): AUC=0.5917, P=0.500, R=0.652, F1=0.566
('brand', 'purpose', 'purpose'): AUC=0.5625, P=0.529, R=0.750, F1=0.621
('brand', 'indications_and_

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.p

('drug', 'dosage_and_administration', 'brand'): AUC=1.0000, P=1.000, R=1.000, F1=1.000
('drug', 'dosage_and_administration', 'storage_and_handling'): AUC=0.1111, P=0.000, R=0.000, F1=0.000
('drug', 'active_ingredient', 'drug_interactions'): AUC=1.0000, P=0.000, R=0.000, F1=0.000
('brand', 'purpose', 'indications_and_usage'): AUC=1.0000, P=1.000, R=1.000, F1=1.000
('drug', 'ask_doctor', 'purpose'): AUC=0.5000, P=0.500, R=1.000, F1=0.667
('drug', 'stop_use', 'ask_doctor'): AUC=0.5200, P=0.000, R=0.000, F1=0.000
('drug', 'keep_out_of_reach_of_children', 'storage_and_handling'): AUC=0.4400, P=0.500, R=0.400, F1=0.444
('drug', 'active_ingredient', 'purpose'): AUC=0.7778, P=1.000, R=0.333, F1=0.500
('drug', 'keep_out_of_reach_of_children', 'pregnancy_or_breast_feeding'): AUC=0.2500, P=0.500, R=0.500, F1=0.500
('drug', 'storage_and_handling', 'dosage_and_administration'): AUC=0.4444, P=0.667, R=0.667, F1=0.667
('drug', 'do_not_use', 'ask_doctor_or_pharmacist'): AUC=0.0000, P=0.000, R=0.000, F

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.p

('drug', 'warnings', 'purpose'): AUC=1.0000, P=0.667, R=1.000, F1=0.800
('drug', 'ask_doctor', 'do_not_use'): AUC=0.0000, P=0.000, R=0.000, F1=0.000
('drug', 'purpose', 'dosage_and_administration'): AUC=0.0000, P=0.000, R=0.000, F1=0.000
('drug', 'purpose', 'active_ingredient'): AUC=0.5000, P=0.500, R=0.500, F1=0.500
('drug', 'ask_doctor', 'dosage_and_administration'): AUC=0.7500, P=0.500, R=0.500, F1=0.500
('drug', 'dosage_and_administration', 'purpose'): AUC=0.7500, P=0.500, R=0.500, F1=0.500
('drug', 'storage_and_handling', 'keep_out_of_reach_of_children'): AUC=0.4444, P=0.000, R=0.000, F1=0.000
('brand', 'indications_and_usage', 'dosage_and_administration'): AUC=0.7500, P=0.667, R=1.000, F1=0.800
('active_ingredient', 'do_not_use', 'warnings'): AUC=1.0000, P=0.500, R=1.000, F1=0.667
('drug', 'active_ingredient', 'dosage_and_administration'): AUC=1.0000, P=1.000, R=1.000, F1=1.000
('drug', 'indications_and_usage', 'active_ingredient'): AUC=0.0000, P=0.333, R=0.500, F1=0.400
('drug',

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# === GraphML → R-GCN (NeighborLoader + AMP + batched inference) ===
# Fixes:
#  - DistMult scoring (relation-aware) in train + eval
#  - Typed negatives sampled *within the batch* (local), avoiding drop
#  - zero_division=0 to silence undefined metric warnings

import os, gc, random
from collections import defaultdict

import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch_geometric.nn import RGCNConv
from torch_geometric.data import Data
from torch_geometric.loader import NeighborLoader
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
import networkx as nx

# ----------------------------
# 0) Repro + Device
# ----------------------------
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gc.collect()
if device.type == "cuda":
    torch.cuda.empty_cache()
print("Using device:", device)

# ----------------------------
# 1) Load GraphML + normalize attrs
# ----------------------------
graph_path = "/content/drive/MyDrive/drug_data_kg_rebuilt_normalized.graphml"
assert os.path.exists(graph_path), f"Path does not exist: {graph_path}"
print(f"📥 Loading GraphML from: {graph_path}")

G0 = nx.read_graphml(graph_path)
G = nx.MultiDiGraph()
G.add_nodes_from(G0.nodes(data=True))
if G0.is_multigraph():
    for u,v,k,d in G0.edges(keys=True, data=True):
        G.add_edge(u,v,key=k, **(d or {}))
else:
    for u,v,d in G0.edges(data=True):
        G.add_edge(u,v, **(d or {}))

for n,data in G.nodes(data=True):
    if "node_type" not in data:
        data["node_type"] = data.get("type", data.get("category", data.get("kind","unknown")))
for u,v,attr in G.edges(data=True):
    if "relation" not in attr:
        attr["relation"] = attr.get("relationship", attr.get("type", attr.get("label","unknown")))

print(f"✅ Loaded graph with {len(G.nodes)} nodes and {len(G.edges)} edges.")

# ----------------------------
# 2) Typed remap + splits
# ----------------------------
node_type_map = defaultdict(list)
node_id_map = {}
for nid,data in G.nodes(data=True):
    ntype = data.get("node_type","unknown")
    idx = len(node_type_map[ntype])
    node_type_map[ntype].append(nid)
    node_id_map[nid] = (ntype, idx)

edge_type_map = defaultdict(list)  # (src_type, rel, dst_type) -> [(src_local, dst_local)]
for s,t,attr in G.edges(data=True):
    if s not in node_id_map or t not in node_id_map: continue
    rel = attr.get("relation","unknown")
    s_t,s_i = node_id_map[s]
    t_t,t_i = node_id_map[t]
    edge_type_map[(s_t, rel, t_t)].append((s_i,t_i))

print(f"✅ Node types: {list(node_type_map.keys())}")
print(f"✅ Total edge types: {len(edge_type_map)}")

# Global ids per (type, local)
type_offsets, global_id_map = {}, {}
offset = 0
for ntype, nodes in node_type_map.items():
    type_offsets[ntype] = offset
    for i in range(len(nodes)):
        global_id_map[(ntype,i)] = offset + i
    offset += len(nodes)
num_nodes = offset

edge_type_keys = list(edge_type_map.keys())         # rel_id -> (src_type, rel, dst_type)
rel2id = {k:i for i,k in enumerate(edge_type_keys)}
num_relations = len(edge_type_keys)

all_src, all_dst, all_rel = [], [], []
edge_type_splits = {}

for etype, pairs in edge_type_map.items():
    s_type,_, t_type = etype
    if len(pairs) < 5:  # keep only relations with minimum support
        continue
    perm = torch.randperm(len(pairs))
    n_tr = int(0.8*len(pairs)); n_va = int(0.1*len(pairs))
    tr = [pairs[i] for i in perm[:n_tr]]
    va = [pairs[i] for i in perm[n_tr:n_tr+n_va]]
    te = [pairs[i] for i in perm[n_tr+n_va:]]

    r = rel2id[etype]
    for s_l, t_l in tr:
        all_src.append(type_offsets[s_type] + s_l)
        all_dst.append(type_offsets[t_type] + t_l)
        all_rel.append(r)

    edge_type_splits[etype] = {
        "val": torch.tensor([[type_offsets[s_type]+s for s,_ in va],
                             [type_offsets[t_type]+t for _,t in va]], dtype=torch.long),
        "test": torch.tensor([[type_offsets[s_type]+s for s,_ in te],
                              [type_offsets[t_type]+t for _,t in te]], dtype=torch.long),
    }

edge_index_cpu = torch.tensor([all_src, all_dst], dtype=torch.long)
edge_type_cpu  = torch.tensor(all_rel, dtype=torch.long)
print(f"Homogeneous nodes: {num_nodes}, edges used for training: {edge_index_cpu.size(1)}")

# For quick type lookup by GLOBAL id (used for local typed negatives)
type_names = sorted(node_type_map.keys())
type2id = {t:i for i,t in enumerate(type_names)}
global_node_type_id = torch.empty(num_nodes, dtype=torch.long)
for tname, nodes in node_type_map.items():
    t_id = type2id[tname]
    base = type_offsets[tname]
    n = len(nodes)
    global_node_type_id[base:base+n] = t_id

# Existing edges per relation (LOCAL to each (src_type,rel,dst_type))
existing_edges_per_rel = {etype: set(pairs) for etype,pairs in edge_type_map.items()}
def relid_to_etype(rel_id:int):
    return edge_type_keys[rel_id]

# ----------------------------
# 3) PyG Data + NeighborLoader
# ----------------------------
data_pyg = Data(num_nodes=num_nodes, edge_index=edge_index_cpu, edge_type=edge_type_cpu)

NUM_NEIGHBORS = [15, 10]
NBATCH_SIZE   = 4096
NUM_WORKERS   = 2  # quiets the OS warning

train_src_unique = torch.unique(edge_index_cpu[0])
train_loader = NeighborLoader(
    data_pyg,
    num_neighbors=NUM_NEIGHBORS,
    batch_size=NBATCH_SIZE,
    input_nodes=train_src_unique,
    shuffle=True,
    num_workers=NUM_WORKERS
)

# ----------------------------
# 4) Model (DistMult) + helpers
# ----------------------------
class RGCN(nn.Module):
    def __init__(self, num_nodes, num_relations, emb_dim=64, hidden_dim=128, out_dim=32, num_bases=30):
        super().__init__()
        self.emb = nn.Embedding(num_nodes, emb_dim)
        self.conv1 = RGCNConv(emb_dim, hidden_dim, num_relations=num_relations, num_bases=num_bases)
        self.conv2 = RGCNConv(hidden_dim, out_dim, num_relations=num_relations, num_bases=num_bases)
        self.rel_emb = nn.Embedding(num_relations, out_dim)

        nn.init.xavier_uniform_(self.emb.weight)
        nn.init.xavier_uniform_(self.rel_emb.weight)

    def forward(self, n_id, edge_index, edge_type):
        # n_id: global ids ordered as local nodes in the subgraph
        x = self.emb(n_id)
        x = self.conv1(x, edge_index, edge_type).relu_()
        x = self.conv2(x, edge_index, edge_type)
        return x  # local node embeddings

    def score(self, h, r, t):
        # DistMult (logits)
        return (h * r * t).sum(dim=-1)

def distmult_sigmoid_scores(z, rel_emb, eidx, rel_ids):
    h = z[eidx[0]]; t = z[eidx[1]]
    r = rel_emb(rel_ids)
    return (h * r * t).sum(dim=-1).sigmoid()

# ----------------------------
# 5) Local (in-batch) typed negatives
# ----------------------------
def build_local_type_pools(n_id_device):
    """Map node_type_id -> list of LOCAL indices present in this batch."""
    g_types = global_node_type_id.to(n_id_device.device, non_blocking=True)[n_id_device]
    pools = defaultdict(list)
    for local_idx, t in enumerate(g_types.tolist()):
        pools[t].append(local_idx)
    for k in list(pools.keys()):
        pools[k] = torch.tensor(pools[k], dtype=torch.long, device=n_id_device.device)
    return pools

def sample_neg_local(pos_local_eidx, rel_ids_local, n_id, pools, k_neg=1):
    """
    Typed negatives *within* the sampled subgraph.
    pos_local_eidx: (2,P) local indices
    rel_ids_local:  (P,)   relation ids (global rel ids)
    pools: dict[type_id] -> local node index tensor present in batch
    """
    if pos_local_eidx.numel() == 0:  # empty
        return torch.empty((2,0), dtype=torch.long, device=pos_local_eidx.device)

    P = pos_local_eidx.size(1)
    neg_src, neg_dst = [], []
    for i in range(P):
        s_l = int(pos_local_eidx[0, i])
        d_l = int(pos_local_eidx[1, i])
        r_id = int(rel_ids_local[i])
        s_type, _, d_type = relid_to_etype(r_id)
        s_tid = type2id[s_type]; d_tid = type2id[d_type]

        # pick from local pools — if missing (rare), fall back to uniform local sampling
        s_pool = pools.get(s_tid, None)
        d_pool = pools.get(d_tid, None)
        if s_pool is None or s_pool.numel() == 0:
            s_pool = torch.arange(n_id.size(0), device=n_id.device)
        if d_pool is None or d_pool.numel() == 0:
            d_pool = torch.arange(n_id.size(0), device=n_id.device)

        for _ in range(max(1, k_neg)):
            if random.random() < 0.5:
                # corrupt head
                new_s_l = s_pool[torch.randint(0, s_pool.numel(), (1,))].item()
                neg_src.append(new_s_l); neg_dst.append(d_l)
            else:
                # corrupt tail
                new_d_l = d_pool[torch.randint(0, d_pool.numel(), (1,))].item()
                neg_src.append(s_l); neg_dst.append(new_d_l)

    if len(neg_src) == 0:
        return torch.empty((2,0), dtype=torch.long, device=n_id.device)
    return torch.stack([torch.tensor(neg_src, device=n_id.device),
                        torch.tensor(neg_dst, device=n_id.device)], dim=0)

# ----------------------------
# 6) Train with NeighborLoader + AMP
# ----------------------------
EPOCHS = 30
LR     = 1e-3
K_NEG  = 1

model = RGCN(num_nodes, num_relations, emb_dim=64, hidden_dim=128, out_dim=32, num_bases=30).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)
scaler = GradScaler(enabled=(device.type == "cuda"))

# Global training edges (for picking positives per batch)
src_glob_all = edge_index_cpu[0].to(device)
dst_glob_all = edge_index_cpu[1].to(device)
rels_all     = edge_type_cpu.to(device)

print("\n🔁 Training with NeighborLoader mini-batches + AMP ...")
for epoch in range(1, EPOCHS+1):
    model.train()
    epoch_loss = 0.0
    seen_pos = 0

    for batch in train_loader:
        batch = batch.to(device, non_blocking=True)
        n_id = batch.n_id  # global ids in this subgraph (device tensor)

        # global->local mapping
        global2local = -torch.ones(num_nodes, dtype=torch.long, device=device)
        global2local[n_id] = torch.arange(n_id.size(0), device=device)

        # positives inside batch
        in_src = global2local[src_glob_all]
        in_dst = global2local[dst_glob_all]
        mask = (in_src >= 0) & (in_dst >= 0)
        pos_idx = torch.nonzero(mask, as_tuple=False).view(-1)
        if pos_idx.numel() == 0:
            del batch, n_id, global2local, in_src, in_dst, mask, pos_idx
            if device.type == "cuda": torch.cuda.empty_cache()
            continue

        MAX_POS_PER_BATCH = 8192
        if pos_idx.numel() > MAX_POS_PER_BATCH:
            pos_idx = pos_idx[torch.randperm(pos_idx.numel(), device=device)[:MAX_POS_PER_BATCH]]

        pos_local = torch.stack([in_src[pos_idx], in_dst[pos_idx]], dim=0)
        rel_ids_local = rels_all[pos_idx]

        # local typed pools for negatives
        pools = build_local_type_pools(n_id)

        optimizer.zero_grad(set_to_none=True)
        with autocast(enabled=(device.type == "cuda")):
            z = model(n_id, batch.edge_index, batch.edge_type)

            # DistMult positives
            pos_h = z[pos_local[0]]; pos_t = z[pos_local[1]]
            pos_r = model.rel_emb(rel_ids_local)
            pos_logits = model.score(pos_h, pos_r, pos_t)

            # local typed negatives
            neg_local = sample_neg_local(pos_local, rel_ids_local, n_id, pools, k_neg=K_NEG)
            if neg_local.size(1) > 0:
                neg_h = z[neg_local[0]]; neg_t = z[neg_local[1]]
                # repeat relation ids for negs
                neg_r = model.rel_emb(rel_ids_local.repeat_interleave(K_NEG))
                neg_logits = model.score(neg_h, neg_r, neg_t)
                loss = (F.binary_cross_entropy_with_logits(pos_logits, torch.ones_like(pos_logits)) +
                        F.binary_cross_entropy_with_logits(neg_logits, torch.zeros_like(neg_logits)))
            else:
                loss = F.binary_cross_entropy_with_logits(pos_logits, torch.ones_like(pos_logits))

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += float(loss.detach().cpu())
        seen_pos += pos_local.size(1)

        # cleanup
        del batch, n_id, global2local, in_src, in_dst, mask, pos_idx
        del pos_local, rel_ids_local, pools, z, pos_h, pos_t, pos_r, pos_logits
        if 'neg_local' in locals(): del neg_local, neg_h, neg_t, neg_r, neg_logits
        if device.type == "cuda": torch.cuda.empty_cache()

    print(f"Epoch {epoch:02d} | Loss: {epoch_loss:.4f} | Seen pos edges: {seen_pos}")

# ----------------------------
# 7) Batched inference (DistMult-ready embeddings)
# ----------------------------
print("\n🔎 Computing full-node embeddings (batched inference) ...")
model.eval()
OUT_DIM = 32
z_all = torch.empty((num_nodes, OUT_DIM), dtype=torch.float32, device=device)

INFER_BATCH = 16384
infer_loader = NeighborLoader(
    Data(num_nodes=num_nodes, edge_index=edge_index_cpu, edge_type=edge_type_cpu),
    num_neighbors=[-1, -1],
    batch_size=INFER_BATCH,
    input_nodes=torch.arange(num_nodes),
    shuffle=False,
    num_workers=NUM_WORKERS
)
with torch.no_grad():
    for batch in infer_loader:
        batch = batch.to(device, non_blocking=True)
        n_id = batch.n_id
        z_batch = model(n_id, batch.edge_index, batch.edge_type)
        z_all[n_id] = z_batch
        del batch, z_batch, n_id
        if device.type == "cuda": torch.cuda.empty_cache()

# ----------------------------
# 8) Evaluation (per relation + overall) — DistMult + zero_division=0
# ----------------------------
print("\n📊 Final evaluation per edge type (filtered negatives, DistMult):")

def eval_relation(z_all, etype, splits, k_neg=1):
    rel_id = rel2id[etype]
    pos_e = splits["test"]
    if pos_e.numel() == 0: return None
    N = pos_e.size(1)
    # Build negatives globally (filtered, typed) just for evaluation
    # (global is fine here; we evaluate on z_all)
    # Reuse earlier sampler but with relation ids vector:
    src_g = pos_e[0].to(device); dst_g = pos_e[1].to(device)
    rel_ids = torch.full((N,), rel_id, dtype=torch.long, device=device)

    # filtered typed negs in GLOBAL space (may include true edges but low prob)
    # Simple version: corrupt tails only
    neg_src = src_g.repeat_interleave(k_neg)
    # build tail pool by type
    s_type,_, d_type = etype
    d_tid = type2id[d_type]
    d_pool_global = torch.arange(type_offsets[d_type], type_offsets[d_type] + len(node_type_map[d_type]), device=device)
    neg_dst = d_pool_global[torch.randint(0, d_pool_global.numel(), (N*k_neg,), device=device)]

    # Scores
    pos_scores = distmult_sigmoid_scores(z_all, model.rel_emb, pos_e.to(device), rel_ids)
    neg_scores = distmult_sigmoid_scores(z_all, model.rel_emb, torch.stack([neg_src, neg_dst]), rel_ids.repeat_interleave(k_neg))

    y_true = torch.cat([torch.ones_like(pos_scores), torch.zeros_like(neg_scores)])
    y_scores = torch.cat([pos_scores, neg_scores]).detach().cpu().numpy()
    y = y_true.detach().cpu().numpy()
    auc = roc_auc_score(y, y_scores)

    preds = (y_scores > 0.5).astype(int)
    p = precision_score(y, preds, zero_division=0)
    r = recall_score(y, preds, zero_division=0)
    f1 = f1_score(y, preds, zero_division=0)
    return auc, p, r, f1

results = []
all_pos_eidx_list, all_rel_ids_list = [], []

for etype, splits in edge_type_splits.items():
    out = eval_relation(z_all, etype, splits, k_neg=1)
    if out is None: continue
    auc, p, r, f1 = out
    results.append((etype, auc, p, r, f1))
    rel_id = rel2id[etype]
    te = splits["test"]
    all_pos_eidx_list.append(te)
    all_rel_ids_list.append(torch.full((te.size(1),), rel_id, dtype=torch.long))
    print(f"{etype}: AUC={auc:.4f}, P={p:.3f}, R={r:.3f}, F1={f1:.3f}")

# Overall
if all_pos_eidx_list:
    all_pos_eidx = torch.cat(all_pos_eidx_list, dim=1).to(device)
    all_rel_ids = torch.cat(all_rel_ids_list, dim=0).to(device)
    # Simple overall negatives: corrupt tails with matching type (1 per pos)
    etypes_for_all = [relid_to_etype(int(r)) for r in all_rel_ids.tolist()]
    neg_src = all_pos_eidx[0]
    neg_dst_list = []
    for r_id, et in zip(all_rel_ids.tolist(), etypes_for_all):
        d_type = et[2]
        pool = torch.arange(type_offsets[d_type], type_offsets[d_type] + len(node_type_map[d_type]), device=device)
        neg_dst_list.append(pool[torch.randint(0, pool.numel(), (1,), device=device)])
    neg_dst = torch.stack(neg_dst_list).view(-1)
    neg_e = torch.stack([neg_src, neg_dst], dim=0)

    pos_scores = distmult_sigmoid_scores(z_all, model.rel_emb, all_pos_eidx, all_rel_ids)
    neg_scores = distmult_sigmoid_scores(z_all, model.rel_emb, neg_e, all_rel_ids)

    y_scores = torch.cat([pos_scores, neg_scores]).detach().cpu().numpy()
    y = np.concatenate([np.ones(pos_scores.numel()), np.zeros(neg_scores.numel())])

    overall_auc = roc_auc_score(y, y_scores)
    preds = (y_scores > 0.5).astype(int)
    overall_p = precision_score(y, preds, zero_division=0)
    overall_r = recall_score(y, preds, zero_division=0)
    overall_f1 = f1_score(y, preds, zero_division=0)

    print("\n🌍 Overall Performance (filtered-ish, typed tails, DistMult):")
    print(f"Overall AUC = {overall_auc:.4f}")
    print(f"Overall Precision = {overall_p:.3f}")
    print(f"Overall Recall = {overall_r:.3f}")
    print(f"Overall F1 Score = {overall_f1:.3f}")
else:
    print("\nNo test edges available for evaluation.")


Using device: cuda
📥 Loading GraphML from: /content/drive/MyDrive/drug_data_kg_rebuilt_normalized.graphml
✅ Loaded graph with 100495 nodes and 160229 edges.
✅ Node types: ['drug', 'active_ingredient', 'indications_and_usage', 'dosage_and_administration', 'warnings', 'pregnancy_or_breast_feeding', 'keep_out_of_reach_of_children', 'purpose', 'storage_and_handling', 'brand', 'do_not_use', 'ask_doctor', 'inactive_ingredient', 'stop_use', 'ask_doctor_or_pharmacist', 'overdose_warning', 'drug_interactions']
✅ Total edge types: 287
Homogeneous nodes: 100495, edges used for training: 127935

🔁 Training with NeighborLoader mini-batches + AMP ...


/usr/local/lib/python3.12/dist-packages/torch_geometric/sampler/neighbor_sampler.py:61: UserWarning: Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated and will be removed soon. Please install 'pyg-lib' for accelerated neighborhood sampling
  warnings.warn(f"Using '{self.__class__.__name__}' without a "
/tmp/ipython-input-3266118914.py:255: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=(device.type == "cuda"))
/tmp/ipython-input-3266118914.py:297: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=(device.type == "cuda")):


Epoch 01 | Loss: 12.4766 | Seen pos edges: 22807
Epoch 02 | Loss: 12.4764 | Seen pos edges: 22725
Epoch 03 | Loss: 12.4751 | Seen pos edges: 22895
Epoch 04 | Loss: 12.4682 | Seen pos edges: 23041
Epoch 05 | Loss: 12.4315 | Seen pos edges: 22789
Epoch 06 | Loss: 12.2815 | Seen pos edges: 23073
Epoch 07 | Loss: 11.7523 | Seen pos edges: 22703
Epoch 08 | Loss: 10.9455 | Seen pos edges: 22849
Epoch 09 | Loss: 10.3098 | Seen pos edges: 23050
Epoch 10 | Loss: 9.8015 | Seen pos edges: 22744
Epoch 11 | Loss: 9.3868 | Seen pos edges: 22859
Epoch 12 | Loss: 8.9740 | Seen pos edges: 23099
Epoch 13 | Loss: 8.6276 | Seen pos edges: 22692
Epoch 14 | Loss: 8.3217 | Seen pos edges: 22912
Epoch 15 | Loss: 8.0881 | Seen pos edges: 22930
Epoch 16 | Loss: 7.7698 | Seen pos edges: 22923
Epoch 17 | Loss: 7.4389 | Seen pos edges: 22892
Epoch 18 | Loss: 7.1008 | Seen pos edges: 22997
Epoch 19 | Loss: 6.6699 | Seen pos edges: 22790
Epoch 20 | Loss: 6.4369 | Seen pos edges: 22882
Epoch 21 | Loss: 6.0767 | Seen 

In [ ]:
import networkx as nx
from collections import Counter, defaultdict
from pathlib import Path
import json
import re

# =========================
# CONFIG: set your paths
# =========================
OPENFDA_PATH = "/Users/ganeshkumarboini/Downloads/drug_data_kg_openfda_edges_no_drop.graphml"
PRIMEKG_PATH = "/Users/ganeshkumarboini/Downloads/primekg_hetionet_combined.graphml"
OUT_MERGED_GRAPH = "/Users/ganeshkumarboini/Downloads/merged_openfda_primekg.graphml"
OUT_SUMMARY_JSON = "/Users/ganeshkumarboini/Downloads/merged_openfda_primekg.summary.json"

# Options
USE_MULTIGRAPH = True          # keep every parallel edge (no drops)
NORMALIZE_NAMES = True         # lower/strip/collapse spaces for node_name when matching
PREFER_NODE_SOURCE = "primekg" # if node attribute conflicts: "openfda" or "primekg"

# =========================
# Helpers
# =========================
def load_graph(path):
    G = nx.read_graphml(path)
    kind = "MultiDiGraph" if G.is_multigraph() else "DiGraph"
    print(f"Loaded {path} -> {kind} | nodes={G.number_of_nodes():,} edges={G.number_of_edges():,}")
    return G

def get_attr(attrs, k, default=""):
    v = attrs.get(k)
    return default if v is None else v

_ws_re = re.compile(r"\s+")
def canon_name(name: str) -> str:
    s = name if isinstance(name, str) else str(name)
    if NORMALIZE_NAMES:
        s = _ws_re.sub(" ", s.strip().lower())
    return s

def node_inventory(G, node_type_key="node_type", node_name_key="node_name"):
    types = Counter(nx.get_node_attributes(G, node_type_key).values())
    names_missing = sum(1 for _, a in G.nodes(data=True) if node_name_key not in a or not str(a.get(node_name_key,"")).strip())
    return types, names_missing

def ensure_node_basics(G, source_tag):
    # guarantee node_name and node_source exist
    for n, a in G.nodes(data=True):
        if "node_name" not in a or not str(a.get("node_name","")).strip():
            a["node_name"] = str(n)
        # stamp node_source if missing (may already exist in your KGs)
        if "node_source" not in a or not a["node_source"]:
            a["node_source"] = source_tag

def summarize_graph(G, node_type_key="node_type", edge_rel_key="relation"):
    kind = "MultiDiGraph" if G.is_multigraph() else "DiGraph"
    n_nodes = G.number_of_nodes()
    n_edges = G.number_of_edges()
    node_types = Counter(nx.get_node_attributes(G, node_type_key).values())
    edge_types = Counter(nx.get_edge_attributes(G, edge_rel_key).values())

    node_types_list = sorted(node_types.items(), key=lambda kv: (-kv[1], kv[0]))
    edge_types_list = sorted(edge_types.items(), key=lambda kv: (-kv[1], kv[0]))

    # connectivity (MultiGraphs supported)
    UG = G.to_undirected()
    comps = list(nx.connected_components(UG))
    largest = max((len(c) for c in comps), default=0)
    frac = (largest / n_nodes) if n_nodes else 0.0
    orphans = sum(1 for _ in nx.isolates(G))

    summary = {
        "graph_kind": kind,
        "nodes": int(n_nodes),
        "edges": int(n_edges),

        "node_types_count": int(len(node_types_list)),
        "node_types": node_types_list,

        "relation_types_count": int(len(edge_types_list)),
        "relation_types": edge_types_list,

        "connectivity": {
            "components": int(len(comps)),
            "largest_component_size": int(largest),
            "largest_component_fraction": float(frac),
            "orphans": int(orphans),
        },

        "keys_used": {
            "node_type_key": node_type_key,
            "edge_rel_key": edge_rel_key,
        },
    }
    return summary

def triplet_schema(G, node_type_key="node_type", edge_rel_key="relation", top=30):
    # list unique (src_type, relation, dst_type) and counts
    counts = Counter()
    # works for DiGraph & MultiDiGraph
    if G.is_multigraph():
        for u, v, k, a in G.edges(keys=True, data=True):
            r = a.get(edge_rel_key, "unknown")
            st = G.nodes[u].get(node_type_key, "unknown")
            dt = G.nodes[v].get(node_type_key, "unknown")
            counts[(st, r, dt)] += 1
    else:
        for u, v, a in G.edges(data=True):
            r = a.get(edge_rel_key, "unknown")
            st = G.nodes[u].get(node_type_key, "unknown")
            dt = G.nodes[v].get(node_type_key, "unknown")
            counts[(st, r, dt)] += 1
    total = len(counts)
    top_items = counts.most_common(top)
    return total, top_items

# =========================
# Load graphs
# =========================
G1 = load_graph(OPENFDA_PATH)     # your normalized OpenFDA minimal (node_type/node_name/node_source + relation/subfield)
G2 = load_graph(PRIMEKG_PATH)     # PrimeKG+Hetionet combined

# Ensure basic attrs present
ensure_node_basics(G1, "openfda")
ensure_node_basics(G2, "primekg")

# Quick pre-merge stats
t1, miss1 = node_inventory(G1)
t2, miss2 = node_inventory(G2)
print("\nOpenFDA types (top):", t1.most_common(10), "| missing node_name:", miss1)
print("PrimeKG+Hetionet types (top):", t2.most_common(10), "| missing node_name:", miss2)

# =========================
# Build canonical map by (node_type, node_name)
# We create a fresh MultiDiGraph (or DiGraph) so nothing is lost.
# =========================
MG = nx.MultiDiGraph() if USE_MULTIGRAPH else nx.DiGraph()

NODE_TYPE_KEY = "node_type"
NODE_NAME_KEY = "node_name"
NODE_SOURCE_KEY = "node_source"
EDGE_REL_KEY = "relation"

# map canonical key -> merged node id
canon2id = {}
# keep an incremental integer suffix to avoid accidental id collisions
next_int = 0

def make_merged_id(ntype, nname):
    # Construct a stable readable id. If collides, append counter.
    base = f"{ntype}::{nname}"
    nonlocal next_int
    if base in MG:  # extremely rare; but handle
        while f"{base}##{next_int}" in MG:
            next_int += 1
        return f"{base}##{next_int}"
    return base

def upsert_node(nid, attrs, source_graph):
    """Insert node into merged graph using canonical (type, name) matching. Returns merged node id."""
    ntype = get_attr(attrs, NODE_TYPE_KEY, "unknown")
    nname = get_attr(attrs, NODE_NAME_KEY, str(nid))
    ckey = (canon_name(ntype), canon_name(nname))

    if ckey in canon2id:
        mid = canon2id[ckey]
        # merge attributes (prefer PRIMEKG if configured)
        existing = MG.nodes[mid]
        # keep node_type and node_name as-is
        # merge node_source
        srcs = set(str(existing.get(NODE_SOURCE_KEY, "")).split("|")) if existing.get(NODE_SOURCE_KEY) else set()
        srcs2 = set(str(attrs.get(NODE_SOURCE_KEY, "")).split("|")) if attrs.get(NODE_SOURCE_KEY) else set()
        all_srcs = [s for s in (srcs | srcs2) if s]
        MG.nodes[mid][NODE_SOURCE_KEY] = "|".join(sorted(set(all_srcs))) if all_srcs else existing.get(NODE_SOURCE_KEY, "")
        # retain an 'orig_id_<source>' list
        tag = "primekg" if source_graph == "primekg" else "openfda"
        orig_key = f"orig_id_{tag}"
        olds = set(str(MG.nodes[mid].get(orig_key, "")).split("|")) if MG.nodes[mid].get(orig_key) else set()
        olds.add(str(nid))
        MG.nodes[mid][orig_key] = "|".join(sorted(olds))
        return mid
    else:
        mid = make_merged_id(ntype, nname)
        canon2id[ckey] = mid
        # initialize node attrs
        MG.add_node(mid)
        MG.nodes[mid][NODE_TYPE_KEY] = ntype
        MG.nodes[mid][NODE_NAME_KEY] = nname
        MG.nodes[mid][NODE_SOURCE_KEY] = attrs.get(NODE_SOURCE_KEY, source_graph)
        # store original ids by source
        tag = "primekg" if source_graph == "primekg" else "openfda"
        MG.nodes[mid][f"orig_id_{tag}"] = str(nid)
        return mid

def add_edge_preserving(u_mid, v_mid, eattrs, source_graph):
    # always keep relation and subfield if present
    rel = eattrs.get(EDGE_REL_KEY, "unknown")
    subf = eattrs.get("subfield", None)
    payload = {
        EDGE_REL_KEY: rel,
        "source_graph": source_graph,
    }
    if subf is not None:
        payload["subfield"] = subf
    # optional: keep original endpoints for traceability
    # payload["orig_u"] = str(u_mid)  # usually not needed
    # payload["orig_v"] = str(v_mid)
    if MG.is_multigraph():
        MG.add_edge(u_mid, v_mid, **payload)
    else:
        # In DiGraph, last write wins; to avoid drops we could count, but we promised no drops, so Multi is recommended.
        MG.add_edge(u_mid, v_mid, **payload)

# =========================
# Add OpenFDA nodes/edges
# =========================
for n, a in G1.nodes(data=True):
    upsert_node(n, a, "openfda")

if G1.is_multigraph():
    for u, v, k, a in G1.edges(keys=True, data=True):
        u_mid = upsert_node(u, G1.nodes[u], "openfda")
        v_mid = upsert_node(v, G1.nodes[v], "openfda")
        add_edge_preserving(u_mid, v_mid, a, "openfda")
else:
    for u, v, a in G1.edges(data=True):
        u_mid = upsert_node(u, G1.nodes[u], "openfda")
        v_mid = upsert_node(v, G1.nodes[v], "openfda")
        add_edge_preserving(u_mid, v_mid, a, "openfda")

print(f"\nAfter OpenFDA -> merged: nodes={MG.number_of_nodes():,} edges={MG.number_of_edges():,}")

# =========================
# Add PrimeKG nodes/edges
# =========================
for n, a in G2.nodes(data=True):
    upsert_node(n, a, "primekg")

if G2.is_multigraph():
    for u, v, k, a in G2.edges(keys=True, data=True):
        u_mid = upsert_node(u, G2.nodes[u], "primekg")
        v_mid = upsert_node(v, G2.nodes[v], "primekg")
        add_edge_preserving(u_mid, v_mid, a, "primekg")
else:
    for u, v, a in G2.edges(data=True):
        u_mid = upsert_node(u, G2.nodes[u], "primekg")
        v_mid = upsert_node(v, G2.nodes[v], "primekg")
        add_edge_preserving(u_mid, v_mid, a, "primekg")

print(f"After PrimeKG -> merged: nodes={MG.number_of_nodes():,} edges={MG.number_of_edges():,}")

# =========================
# Optional: prefer one source's attrs when conflicts (kept minimal here)
# =========================
# You can extend this block to pull preferred per-type attributes.

# =========================
# Save merged graph
# =========================
nx.write_graphml(MG, OUT_MERGED_GRAPH)
print(f"\nSaved merged graph -> {OUT_MERGED_GRAPH}")

# =========================
# Build & save summary
# =========================
summary = summarize_graph(MG, node_type_key=NODE_TYPE_KEY, edge_rel_key=EDGE_REL_KEY)

# Triplet schema preview
schema_total, schema_top = triplet_schema(MG, node_type_key=NODE_TYPE_KEY, edge_rel_key=EDGE_REL_KEY, top=30)
summary["triplet_schema_types"] = int(schema_total)
summary["triplet_schema_top"] = [[list(k), int(v)] for k, v in schema_top]

with open(OUT_SUMMARY_JSON, "w") as f:
    json.dump(summary, f, indent=2)

print("\n=== MERGED SUMMARY ===")
print(json.dumps(summary, indent=2))


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ganeshkumarboini/Downloads/drug_data_kg_openfda_edges_no_drop.graphml'

In [ ]:
import networkx as nx
from collections import Counter, defaultdict
from pathlib import Path
import json
import re
import pickle
from google.colab import drive

# =========================
# MOUNT GOOGLE DRIVE
# =========================
drive.mount('/content/drive')

# =========================
# CONFIG: set your paths (update these to your file locations in Drive)
# =========================
# Update these paths to point to your files in Google Drive
OPENFDA_PATH = "/content/drive/MyDrive/drug_data_kg_openfda_edges_no_drop.graphml"
PRIMEKG_PATH = "/content/drive/MyDrive/primekg_hetionet_combined.graphml"
OUT_MERGED_GRAPH = "/content/drive/MyDrive/merged_openfda_primekg.graphml"
OUT_MERGED_PICKLE = "/content/drive/MyDrive/merged_openfda_primekg.pickle"
OUT_SUMMARY_JSON = "/content/drive/MyDrive/merged_openfda_primekg.summary.json"

# Options
USE_MULTIGRAPH = True          # keep every parallel edge (no drops)
NORMALIZE_NAMES = True         # lower/strip/collapse spaces for node_name when matching
PREFER_NODE_SOURCE = "primekg" # if node attribute conflicts: "openfda" or "primekg"

# =========================
# Helpers
# =========================
def load_graph(path):
    G = nx.read_graphml(path)
    kind = "MultiDiGraph" if G.is_multigraph() else "DiGraph"
    print(f"Loaded {path} -> {kind} | nodes={G.number_of_nodes():,} edges={G.number_of_edges():,}")
    return G

def get_attr(attrs, k, default=""):
    v = attrs.get(k)
    return default if v is None else v

_ws_re = re.compile(r"\s+")
def canon_name(name: str) -> str:
    s = name if isinstance(name, str) else str(name)
    if NORMALIZE_NAMES:
        s = _ws_re.sub(" ", s.strip().lower())
    return s

def node_inventory(G, node_type_key="node_type", node_name_key="node_name"):
    types = Counter(nx.get_node_attributes(G, node_type_key).values())
    names_missing = sum(1 for _, a in G.nodes(data=True) if node_name_key not in a or not str(a.get(node_name_key,"")).strip())
    return types, names_missing

def ensure_node_basics(G, source_tag):
    # guarantee node_name and node_source exist
    for n, a in G.nodes(data=True):
        if "node_name" not in a or not str(a.get("node_name","")).strip():
            a["node_name"] = str(n)
        # stamp node_source if missing (may already exist in your KGs)
        if "node_source" not in a or not a["node_source"]:
            a["node_source"] = source_tag

def summarize_graph(G, node_type_key="node_type", edge_rel_key="relation"):
    kind = "MultiDiGraph" if G.is_multigraph() else "DiGraph"
    n_nodes = G.number_of_nodes()
    n_edges = G.number_of_edges()
    node_types = Counter(nx.get_node_attributes(G, node_type_key).values())
    edge_types = Counter(nx.get_edge_attributes(G, edge_rel_key).values())

    node_types_list = sorted(node_types.items(), key=lambda kv: (-kv[1], kv[0]))
    edge_types_list = sorted(edge_types.items(), key=lambda kv: (-kv[1], kv[0]))

    # connectivity (MultiGraphs supported)
    UG = G.to_undirected()
    comps = list(nx.connected_components(UG))
    largest = max((len(c) for c in comps), default=0)
    frac = (largest / n_nodes) if n_nodes else 0.0
    orphans = sum(1 for _ in nx.isolates(G))

    summary = {
        "graph_kind": kind,
        "nodes": int(n_nodes),
        "edges": int(n_edges),

        "node_types_count": int(len(node_types_list)),
        "node_types": node_types_list,

        "relation_types_count": int(len(edge_types_list)),
        "relation_types": edge_types_list,

        "connectivity": {
            "components": int(len(comps)),
            "largest_component_size": int(largest),
            "largest_component_fraction": float(frac),
            "orphans": int(orphans),
        },

        "keys_used": {
            "node_type_key": node_type_key,
            "edge_rel_key": edge_rel_key,
        },
    }
    return summary

def triplet_schema(G, node_type_key="node_type", edge_rel_key="relation", top=30):
    # list unique (src_type, relation, dst_type) and counts
    counts = Counter()
    # works for DiGraph & MultiDiGraph
    if G.is_multigraph():
        for u, v, k, a in G.edges(keys=True, data=True):
            r = a.get(edge_rel_key, "unknown")
            st = G.nodes[u].get(node_type_key, "unknown")
            dt = G.nodes[v].get(node_type_key, "unknown")
            counts[(st, r, dt)] += 1
    else:
        for u, v, a in G.edges(data=True):
            r = a.get(edge_rel_key, "unknown")
            st = G.nodes[u].get(node_type_key, "unknown")
            dt = G.nodes[v].get(node_type_key, "unknown")
            counts[(st, r, dt)] += 1
    total = len(counts)
    top_items = counts.most_common(top)
    return total, top_items

# =========================
# Load graphs
# =========================
print("Loading graphs...")
G1 = load_graph(OPENFDA_PATH)     # your normalized OpenFDA minimal (node_type/node_name/node_source + relation/subfield)
G2 = load_graph(PRIMEKG_PATH)     # PrimeKG+Hetionet combined

# Ensure basic attrs present
ensure_node_basics(G1, "openfda")
ensure_node_basics(G2, "primekg")

# Quick pre-merge stats
t1, miss1 = node_inventory(G1)
t2, miss2 = node_inventory(G2)
print("\nOpenFDA types (top):", t1.most_common(10), "| missing node_name:", miss1)
print("PrimeKG+Hetionet types (top):", t2.most_common(10), "| missing node_name:", miss2)

# =========================
# Build canonical map by (node_type, node_name)
# We create a fresh MultiDiGraph (or DiGraph) so nothing is lost.
# =========================
MG = nx.MultiDiGraph() if USE_MULTIGRAPH else nx.DiGraph()

NODE_TYPE_KEY = "node_type"
NODE_NAME_KEY = "node_name"
NODE_SOURCE_KEY = "node_source"
EDGE_REL_KEY = "relation"

# map canonical key -> merged node id
canon2id = {}
# keep an incremental integer suffix to avoid accidental id collisions
next_int = 0

def make_merged_id(ntype, nname):
    # Construct a stable readable id. If collides, append counter.
    base = f"{ntype}::{nname}"
    nonlocal next_int
    if base in MG:  # extremely rare; but handle
        while f"{base}##{next_int}" in MG:
            next_int += 1
        return f"{base}##{next_int}"
    return base

def upsert_node(nid, attrs, source_graph):
    """Insert node into merged graph using canonical (type, name) matching. Returns merged node id."""
    ntype = get_attr(attrs, NODE_TYPE_KEY, "unknown")
    nname = get_attr(attrs, NODE_NAME_KEY, str(nid))
    ckey = (canon_name(ntype), canon_name(nname))

    if ckey in canon2id:
        mid = canon2id[ckey]
        # merge attributes (prefer PRIMEKG if configured)
        existing = MG.nodes[mid]
        # keep node_type and node_name as-is
        # merge node_source
        srcs = set(str(existing.get(NODE_SOURCE_KEY, "")).split("|")) if existing.get(NODE_SOURCE_KEY) else set()
        srcs2 = set(str(attrs.get(NODE_SOURCE_KEY, "")).split("|")) if attrs.get(NODE_SOURCE_KEY) else set()
        all_srcs = [s for s in (srcs | srcs2) if s]
        MG.nodes[mid][NODE_SOURCE_KEY] = "|".join(sorted(set(all_srcs))) if all_srcs else existing.get(NODE_SOURCE_KEY, "")
        # retain an 'orig_id_<source>' list
        tag = "primekg" if source_graph == "primekg" else "openfda"
        orig_key = f"orig_id_{tag}"
        olds = set(str(MG.nodes[mid].get(orig_key, "")).split("|")) if MG.nodes[mid].get(orig_key) else set()
        olds.add(str(nid))
        MG.nodes[mid][orig_key] = "|".join(sorted(olds))
        return mid
    else:
        mid = make_merged_id(ntype, nname)
        canon2id[ckey] = mid
        # initialize node attrs
        MG.add_node(mid)
        MG.nodes[mid][NODE_TYPE_KEY] = ntype
        MG.nodes[mid][NODE_NAME_KEY] = nname
        MG.nodes[mid][NODE_SOURCE_KEY] = attrs.get(NODE_SOURCE_KEY, source_graph)
        # store original ids by source
        tag = "primekg" if source_graph == "primekg" else "openfda"
        MG.nodes[mid][f"orig_id_{tag}"] = str(nid)
        return mid

def add_edge_preserving(u_mid, v_mid, eattrs, source_graph):
    # always keep relation and subfield if present
    rel = eattrs.get(EDGE_REL_KEY, "unknown")
    subf = eattrs.get("subfield", None)
    payload = {
        EDGE_REL_KEY: rel,
        "source_graph": source_graph,
    }
    if subf is not None:
        payload["subfield"] = subf
    # optional: keep original endpoints for traceability
    # payload["orig_u"] = str(u_mid)  # usually not needed
    # payload["orig_v"] = str(v_mid)
    if MG.is_multigraph():
        MG.add_edge(u_mid, v_mid, **payload)
    else:
        # In DiGraph, last write wins; to avoid drops we could count, but we promised no drops, so Multi is recommended.
        MG.add_edge(u_mid, v_mid, **payload)

# =========================
# Add OpenFDA nodes/edges
# =========================
print("Adding OpenFDA nodes and edges...")
for n, a in G1.nodes(data=True):
    upsert_node(n, a, "openfda")

if G1.is_multigraph():
    for u, v, k, a in G1.edges(keys=True, data=True):
        u_mid = upsert_node(u, G1.nodes[u], "openfda")
        v_mid = upsert_node(v, G1.nodes[v], "openfda")
        add_edge_preserving(u_mid, v_mid, a, "openfda")
else:
    for u, v, a in G1.edges(data=True):
        u_mid = upsert_node(u, G1.nodes[u], "openfda")
        v_mid = upsert_node(v, G1.nodes[v], "openfda")
        add_edge_preserving(u_mid, v_mid, a, "openfda")

print(f"After OpenFDA -> merged: nodes={MG.number_of_nodes():,} edges={MG.number_of_edges():,}")

# =========================
# Add PrimeKG nodes/edges
# =========================
print("Adding PrimeKG nodes and edges...")
for n, a in G2.nodes(data=True):
    upsert_node(n, a, "primekg")

if G2.is_multigraph():
    for u, v, k, a in G2.edges(keys=True, data=True):
        u_mid = upsert_node(u, G2.nodes[u], "primekg")
        v_mid = upsert_node(v, G2.nodes[v], "primekg")
        add_edge_preserving(u_mid, v_mid, a, "primekg")
else:
    for u, v, a in G2.edges(data=True):
        u_mid = upsert_node(u, G2.nodes[u], "primekg")
        v_mid = upsert_node(v, G2.nodes[v], "primekg")
        add_edge_preserving(u_mid, v_mid, a, "primekg")

print(f"After PrimeKG -> merged: nodes={MG.number_of_nodes():,} edges={MG.number_of_edges():,}")

# =========================
# Optional: prefer one source's attrs when conflicts (kept minimal here)
# =========================
# You can extend this block to pull preferred per-type attributes.

# =========================
# Save merged graph in multiple formats
# =========================
print("Saving merged graph...")

# Save as GraphML
nx.write_graphml(MG, OUT_MERGED_GRAPH)
print(f"Saved merged graph (GraphML) -> {OUT_MERGED_GRAPH}")

# Save as pickle
with open(OUT_MERGED_PICKLE, 'wb') as f:
    pickle.dump(MG, f, protocol=pickle.HIGHEST_PROTOCOL)
print(f"Saved merged graph (Pickle) -> {OUT_MERGED_PICKLE}")

# =========================
# Build & save summary
# =========================
print("Generating summary...")
summary = summarize_graph(MG, node_type_key=NODE_TYPE_KEY, edge_rel_key=EDGE_REL_KEY)

# Triplet schema preview
schema_total, schema_top = triplet_schema(MG, node_type_key=NODE_TYPE_KEY, edge_rel_key=EDGE_REL_KEY, top=30)
summary["triplet_schema_types"] = int(schema_total)
summary["triplet_schema_top"] = [[list(k), int(v)] for k, v in schema_top]

with open(OUT_SUMMARY_JSON, "w") as f:
    json.dump(summary, f, indent=2)

print("\n=== MERGED SUMMARY ===")
print(json.dumps(summary, indent=2))
print(f"\nSummary saved -> {OUT_SUMMARY_JSON}")

# =========================
# Additional info about the saved files
# =========================
print(f"\n=== FILES SAVED ===")
print(f"GraphML: {OUT_MERGED_GRAPH}")
print(f"Pickle:  {OUT_MERGED_PICKLE}")
print(f"Summary: {OUT_SUMMARY_JSON}")

# Quick comparison of file sizes
import os
if os.path.exists(OUT_MERGED_GRAPH) and os.path.exists(OUT_MERGED_PICKLE):
    graphml_size = os.path.getsize(OUT_MERGED_GRAPH) / (1024*1024)  # MB
    pickle_size = os.path.getsize(OUT_MERGED_PICKLE) / (1024*1024)  # MB
    print(f"\nFile sizes:")
    print(f"GraphML: {graphml_size:.2f} MB")
    print(f"Pickle:  {pickle_size:.2f} MB")
    print(f"Ratio:   {graphml_size/pickle_size:.2f}x (Pickle is {pickle_size/graphml_size:.2f}x smaller)")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading graphs...
Loaded /content/drive/MyDrive/drug_data_kg_openfda_edges_no_drop.graphml -> DiGraph | nodes=100,495 edges=160,229
